## **Step 1: Install Required Libraries and Download the Model**

In [1]:
!pip install llama-cpp-python --upgrade

In [2]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import hf_hub_download

# This will download and return the path to the file
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    local_dir="models"
)

print("Model downloaded to:", model_path)


Model downloaded to: models\mistral-7b-instruct-v0.1.Q4_K_M.gguf


## **Step 2: Load and Clean the Dataset**

In [4]:

import pandas as pd

# Load the dataset
df = pd.read_csv("../Dataset/Averages in in General Aptitude Tests aa.csv" )

# Now we will remove all rows that have any missing values
df_cleaned = df.dropna()  # dropna() removes any row that contains a missing value

# After removing, We need to check if there are still any missing values
print("Missing Values After Removal:\n", df_cleaned.isnull().sum())

# Also, We will check the number of rows left after removing missing values
print(f"Number of rows before cleaning: {df.shape[0]}")
print(f"Number of rows after cleaning: {df_cleaned.shape[0]}")
# Remove the "نوع_الاختبار" column as it contains only one unique value
df_cleaned = df_cleaned.drop(columns=["نوع_الاختبار"])

# Check the first few rows to confirm the column is removed
df_cleaned.head()
# Display the top 5 rows
df_cleaned.head()


Missing Values After Removal:
 اسم_المدرسة                        0
المنطقة_الإدارية                   0
الإدارة_التعليمية                  0
المكتب_التعليمي                    0
السلطة                             0
نوع_التعليم                        0
الجنس                              0
نوع_الاختبار                       0
تخصص_الاختبار                      0
متوسط_أداء_الطلبة_في_المدرسة       0
ترتيب_المدرسة_على_مستوى_المدارس    0
dtype: int64
Number of rows before cleaning: 6720
Number of rows after cleaning: 6706


,اسم_المدرسة,المنطقة_الإدارية,الإدارة_التعليمية,المكتب_التعليمي,السلطة,نوع_التعليم,الجنس,تخصص_الاختبار,متوسط_أداء_الطلبة_في_المدرسة,ترتيب_المدرسة_على_مستوى_المدارس
0,أم سلمة الثانوية للموهوبات - مقررات,مكة المكرمة,الإدارة العامة للتعليم بمنطقة مكة المكرمة,مكتب التعليم جنوب مكة المكرمة,حكومي,تعليم عام بنات,بنات,علمي,89.622377,1
1,ثانوية الرواد الأهلية ( الازدهار ) - مقررات,الرياض,الإدارة العامة للتعليم بمنطقة الرياض,شمال,أهلي,تعليم عام بنات,بنات,نظري,84.440000,1
2,ثانوية الموهوبات ( نظام مقررات ),مكة المكرمة,الإدارة العامة للتعليم بمحافظة جدة,مكتب التعليم بشمال جدة,حكومي,تعليم عام بنات,بنات,علمي,89.280423,2
3,الثانوية السادسة لتحفيظ القرآن الكريم 6 - مقررات,الرياض,الإدارة العامة للتعليم بمنطقة الرياض,شمال,حكومي,تحفيظ بنات,بنات,نظري,83.550000,2
4,ثانوية مدارس الظهران الاهلية (مقررات),الشرقية,الإدارة العامة للتعليم بالمنطقة الشرقية,مكتب التعليم بمحافظة الخبر,أهلي,تعليم عام بنات,بنات,علمي,88.903225,3


## **Step 3: Load the LLaMA Language Model**

In [5]:

from llama_cpp import Llama  # Import the model class

llm = Llama(
    model_path="models/mistral-7b-instruct-v0.1.Q4_K_M.gguf",  # Path to the model file
    n_ctx=2048,    # Context length
    n_threads=4,   # Number of CPU threads
    verbose=True   # Print detailed logs
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 lla

## **Step 4: Define Model Response Function**

In [6]:
# Send prompt to the model and get a response
def ask_model(prompt):
    response = llm(
        prompt=f"[INST] {prompt} [/INST]",  # Format prompt
        max_tokens=1024,  # Max length of output
        temperature=0.7,  # Randomness level
        stop=["</s>"]     # Stop token
    )
    return response["choices"][0]["text"].strip()  # Return clean text



## **Step 5: Define Filtering Logic for User Preferences**

In [7]:
# Get all unique values for filterable columns

regions = df_cleaned["المنطقة_الإدارية"].unique().tolist()
authorities = df_cleaned["السلطة"].unique().tolist()
genders = df_cleaned["الجنس"].unique().tolist()
edu_types = df_cleaned["نوع_التعليم"].unique().tolist()
districts = df_cleaned["الإدارة_التعليمية"].unique().tolist()
offices = df_cleaned["المكتب_التعليمي"].unique().tolist()

# Extract filters based on user's input
def extract_filters(user_input):
    filters = {}
    
 # Check if any option exists in the input text
    def partial_match(text, options):
        for option in options:
            if option in text:
                return option
        return None
    
    # Map each column to its values
    filters_map = {
        "المنطقة_الإدارية": regions,
        "السلطة": authorities,
        "الجنس": genders,
        "نوع_التعليم": edu_types,
        "الإدارة_التعليمية": districts,
        "المكتب_التعليمي": offices
    }
    
    # Loop through the map and find matches in the input
    for column, values in filters_map.items():
        match = partial_match(user_input, values)
        if match:
            filters[column] = match

    return filters

## **Step 6: Apply Filters and Select Top Schools**

In [8]:
# Sample input from the user
user_input = "طالبة في الرياض، تعليم عام، حكومية، بنات، مكتب التعليم شمال الرياض"


# Extract relevant filters from the input
filters = extract_filters(user_input)
print("الفلاتر المستخرجة:", filters)

# Filter the dataset using the extracted filters
filtered_df = df_cleaned.copy()
for col, value in filters.items():
    filtered_df = filtered_df[filtered_df[col] == value]
    
# Get top 3 schools based on best ranking (lowest number = best)
top_schools = filtered_df.sort_values(by="ترتيب_المدرسة_على_مستوى_المدارس", ascending=True).head(3)


# Prepare selected school info to be passed into the prompt
school_info = top_schools[
    ["اسم_المدرسة", "المنطقة_الإدارية", "نوع_التعليم", "ترتيب_المدرسة_على_مستوى_المدارس", "متوسط_أداء_الطلبة_في_المدرسة"]
].to_dict(orient="records")


الفلاتر المستخرجة: {'المنطقة_الإدارية': 'الرياض', 'السلطة': 'حكومي', 'الجنس': 'بنات', 'المكتب_التعليمي': 'شمال'}


## **Step 7: Create the Prompts to Send to the Model**

In [9]:
# Generate the prompt that will be sent to the language model
def template_1(user_input, schools):
    # Start with the student description
    prompt = f"وصف الطالبة:\n{user_input}\n\n"

    # Add the list of top 3 recommended schools (names only)
    prompt += "قائمة المدارس المرشحة:\n\n"
    for i, school in enumerate(schools[:3], 1):
        prompt += f"{i}. {school['اسم_المدرسة']}\n"

    # Final instruction to guide the response
    prompt += (
        "\nمن بين هذه المدارس، ما الأنسب للطالبة؟\n"
        " اكتب:\n"
        "- اسم المدرسة فقط.\n"
        "- السبب باختصار في سطر.\n"
        " الرد يكون باللغة العربية الفصحى فقط، كل معلومة في سطر مختلف، بدون تكرار أو إعادة ذكر القائمة.\n"
    )

    return prompt


In [9]:
# Generate the prompt that will be sent to the language model
def template_2(user_input, schools):
    prompt = f" وصف الطالبة:\n{user_input}\n\n"
    prompt += " قائمة المدارس المرشحة:\n"

    # Loop through the top 3 schools
    for i, school in enumerate(schools[:3], 1):
        prompt += (
            f"{i}. اسم المدرسة: {school['اسم_المدرسة']}\n"
            f"   المنطقة: {school['المنطقة_الإدارية']}\n"
            f"   نوع التعليم: {school['نوع_التعليم']}\n"
            f"   ترتيب المدرسة: {school['ترتيب_المدرسة_على_مستوى_المدارس']}\n\n"
            f"   متوسط الأداء الأكاديمي: {school['متوسط_أداء_الطلبة_في_المدرسة']:.2f}\n\n"

        )

    # Final instruction to the language model
    prompt += (
        "اختر المدرسة صاحبة الترتيب الأكاديمي الأفضل (أي أصغر رقم).\n"
        "اكتب:\n"
        "- اسم المدرسة\n"
        "- الترتيب\n"
        "- متوسط الأداء الأكاديمي\n"
        "- سبب الاختيار بناءً على وصف الطالبة\n"
        " الرد يكون بالعربية الفصحى فقط، بدون تكرار أو إعادة القائمة.\n"
    )



    return prompt

## **Step 8: Send Prompts to the Model and Display Response**

In [11]:
# Generate the prompt using the student's input and top schools
prompt1 = template_1(user_input, school_info)

# Send the prompt to the LLaMA model to get a response
response_1 = ask_model(prompt1)

# Display the model's response
print("=== TEMPLATE 1 RESPONSE ===\n", response_1)

llama_perf_context_print:        load time =   35705.52 ms
llama_perf_context_print: prompt eval time =   35704.54 ms /   372 tokens (   95.98 ms per token,    10.42 tokens per second)
llama_perf_context_print:        eval time =   20064.21 ms /   107 runs   (  187.52 ms per token,     5.33 tokens per second)
llama_perf_context_print:       total time =   55841.33 ms /   479 tokens


=== TEMPLATE 1 RESPONSE ===
 - الثانوية السادسة لتحفيظ القرءان الكريم 6 - مقررات
- ثانوية جامعة الأميرة نورة - مقررات
- الثانوية الخامسة والعشرون 25 - مقررات


In [15]:
# Generate the prompt using the student's input and top schools
prompt2 = template_2(user_input, school_info)

# Send the prompt to the LLaMA model to get a response
response_2 = ask_model(prompt2)

# Display the model's response
print("=== TEMPLATE 2 RESPONSE ===\n", response_2)


Llama.generate: 677 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   74323.92 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   92526.73 ms /   352 runs   (  262.86 ms per token,     3.80 tokens per second)
llama_perf_context_print:       total time =   93010.22 ms /   353 tokens


=== TEMPLATE 2 RESPONSE ===
 رد:
- اسم المدرسة: الثانوية السادسة لتحفيظ القرءان الكريم 6 - مقررات
- الترتيب: 2
- متوسط الأداء الأكاديمي: 83.55

رد:
- اسم المدرسة: ثانوية جامعة الأميرة نورة - مقررات
- الترتيب: 14
- متوسط الأداء الأكاديمي: 83.34

رد:
- اسم المدرسة: الثانوية الخامسة والعشرون 25- مقررات
- الترتيب: 41
- متوسط الأداء الأكاديمي: 81.15

رد:
- بناءً على وصف الطالبة في الرياض، تعليم عام، حكومية، بنات، مكتب التعليم شمال الرياض.
